---
title: "The Power of Data Simulation"
subtitle: "Using Python's NumPy Random Generator to Model Randomness & Build Intuition"
author: "Russ Zaliznyak <[rzaliznyak@gmail.com](mailto:rzaliznyak@gmail.com)>"
date: "2024-11-04"
execute:
  echo: false
format: 
  html: 
    css: style.css
    toc: true
    toc-expand: true
    toc-indent: 1em
jupyter: python3
---



# Introduction

Simulating data is an indispensable tool in the business and data science community. It offers a cost-effective way to model complex relationships and rare events within datasets, enabling the exploration of various scenarios without real-world consequences.

Simulations can facilitate the measurement of competing methods' performance on shared datasets and assess how statistical models perform when the ground truth is known. This helps to refine analytical methodologies and even support training of machine learning models in the absence of real data.

Using website conversion as our example, we are going to simulate huge datasets and build our intuition for the randomness we might expect to see. By the end of this paper, you'll be able to model randomness and analyze results for rate-based metrics on your own.

# Website Conversion Example

Imagine your website converts 10% of its visitors to paying customers. Over the course of 10,000 visitors, how many converts should we expect? A quick back of the envelope calculation shows we should expect 1,000 conversions:
$$
10000 \, \text{visitors} * \frac{10 \, \text{conversions}}{100 \, \text{visitors}} = 1000 \, \text{conversions}
$$


_How much could that number vary on any given day?_

# Simulating Data
Let's simulate the process of website conversion 1,000,000x with n = 10,000 and p = 0.10 (conversion rate).


In [ ]:
#| code-fold: false
#| echo: true
import timeit
from numpy.random import binomial, seed
seed(10)

conversion_rate = 0.10
number_of_visitors = int(1e4)


start = timeit.default_timer()
number_simulated_experiments = int(1e6)
control_simulated_results = binomial(
    number_of_visitors, conversion_rate, size=number_simulated_experiments
)
stop = timeit.default_timer()
print(f"simulation runtime: {round(stop-start,3)} secs")


## Graph Simulation Results

At the end of each simulation, we record and plot how many conversions took place. Each panel represents the results of plotting more simulations together.

Our first simulation generated only `{python} control_simulated_results[0]` conversions. But with enough simulations plotted together, are you seeing a pattern develop?


In [ ]:
#| label: fig-one-plot
#| fig-cap: 'Histograms of the aforementioned simulations. With enougn simulations, it''s possible to see website conversion forms a predictable pattern.'
#| code-fold: true
#| echo: true

import plotly.subplots as sp
import pandas as pd
import plotly.express as px
from math import ceil
from numpy import where, linspace, min, max, cumsum, percentile, array, mean


df = pd.DataFrame(control_simulated_results, columns=["value"])
running_total_df = df.copy()

# Count occurrences and create a running total
running_total_df["count"] = running_total_df["value"].map(
    running_total_df["value"].value_counts()
)
running_total_df["running_total"] = running_total_df.groupby("value").cumcount() + 1

# Rearranging to match the running total of occurrences
running_total_df = running_total_df[["value", "count", "running_total"]]



number_experiments_list = [
    1,
    50,
    200,
    400,
    800,
    2500,
    int(1e4),
    int(1e5),
]
subplot_titles = tuple([f"# sims = {value}" for value in number_experiments_list])
number_experiments_list_length = len(number_experiments_list)


number_cols = 4
number_rows = ceil(number_experiments_list_length / number_cols)
BLUE = "#82b2e0"
RED = "#bd0707"
fig = sp.make_subplots(
    rows=number_rows,
    cols=number_cols,
    subplot_titles=subplot_titles,
    vertical_spacing=0.1,
)

for j in range(len(number_experiments_list)):
    number = number_experiments_list[j]
    fig_0 = px.scatter(
        running_total_df[0:number],
        x="value",
        y="running_total",
        color_discrete_sequence = [BLUE]
    )
    

    row = 1 + int(j / number_cols)
    col = 1 + j - (row - 1) * number_cols
    fig.add_trace(fig_0.data[0], row=row, col=col)


x_ticks = [950, 1000, 1050]
for j in range(len(number_experiments_list)):
    row = 1 + int(j / number_cols)
    col = 1 + j - (row - 1) * number_cols

    fig.update_xaxes(
        #tickvals=x_ticks if j > 0 else [int(df["value"].iloc[0])], row=row, col=col
    )

fig.update_layout(height=600, hovermode=False)
fig.show()

<!--
The cumulative distribution function (CDF) is defined as:
$$
P(X \leq k) = \sum_{i=0}^{k} \binom{n}{i} p^i (1-p)^{n-i}
$$
-->


## Use Simulation Results to Estimate Randomness 

1. What's the chance of 950 conversions or fewer?
2. What's the chance of 1050 conversions or more?
3. What's the chance of between 950 and 1050 conversions (exclusive)?


In [ ]:
#| code-fold: true
#| echo: false
#| label: fig-two-plot
#| fig-cap: The red area is the share of simulations falling into the given range.
import plotly.graph_objecgrts as go
from scipy import stats

standard_error = (conversion_rate*(1-conversion_rate)/number_of_visitors)**0.5
BLUE = "#0177c9"
RED = "#bd0707"
number_points = 250
pdf = stats.gaussian_kde(control_simulated_results)
X = linspace(
    min(control_simulated_results), max(control_simulated_results), number_points
)
estimated_density = pdf.evaluate(X)
cumulative_density = cumsum(estimated_density)
cumulative_density /= cumulative_density[-1]
cdf_list = list(cumulative_density)
fx_list = estimated_density

hover_string = "True Baseline"
is_rate = False
text_list = [
    f"Pr( {hover_string} > { round((100 if is_rate else 1)*X[j],3 if is_rate else 2) }{'%' if is_rate else ''}) = {'{0:.2%}'.format(1 - cdf_list[j])}"
    for j in range(len(X))
]
distribution_titles = (
        "950 conversions or less",
        "1050 conversions or more",
        "Between 950 and 1050 conversions",
    )
distribution_figure = sp.make_subplots(
    rows=1,
    cols=3,
    subplot_titles=distribution_titles,
    vertical_spacing=0.1,
)

lower_bound_list = [950, None, 950]
upper_bound_list = [None, 1050, 1050]


line_color_left = BLUE
line_color_right = BLUE
probability_list = []
for j in range(3):
    lower_bound_position = 0
    upper_bound_position = number_points - 1
    lower_bound = lower_bound_list[j]
    upper_bound = upper_bound_list[j]

    if lower_bound is not None:
        lower_bound_position = where(array(X) > lower_bound)[0][0]
    if upper_bound is not None:
        upper_bound_position = where(array(X) > upper_bound)[0][0]

    prob = None
    probability = None
    if lower_bound is not None and upper_bound is not None:
        line_color_left = BLUE
        line_color_right = BLUE
        prob = mean(
            where(
                (control_simulated_results > lower_bound)
                & (control_simulated_results < upper_bound),
                1,
                0,
            )
        )
        x_axis_title = f"{prob:.2%} of outcomes shaded"
        probability = stats.norm.cdf(
            upper_bound / number_of_visitors, loc=conversion_rate, scale=standard_error
        ) - stats.norm.cdf(
            lower_bound / number_of_visitors, loc=conversion_rate, scale=standard_error
        )
    elif lower_bound is not None and upper_bound is None:
        line_color_left = RED
        prob = mean(where(control_simulated_results <= lower_bound, 1, 0))
        x_axis_title = f"{prob:.2%} of outcomes shaded"

        probability = stats.norm.cdf(
            lower_bound / number_of_visitors, loc=conversion_rate, scale=standard_error
        )
    elif upper_bound is not None and lower_bound is None:
        line_color_right = RED
        prob = mean(where(control_simulated_results >= upper_bound, 1, 0))
        x_axis_title = f"{prob:.2%} of outcomes shaded"
        probability = 1 - stats.norm.cdf(
            upper_bound / number_of_visitors, loc=conversion_rate, scale=standard_error
        )
    else:
        pass
    probability_list.append(probability)
    x_axis_title = f"{prob:.2%} of outcomes shaded" if prob is not None else None

    line_color_middle = RED if line_color_left == BLUE else BLUE
    distribution_figure.add_trace(
        go.Scatter(
            x=X[0:lower_bound_position],
            y=fx_list[0:lower_bound_position],
            hoverinfo="skip",
            # marker_symbol="square",
            name="< 0 ",
            marker=dict(size=50),
            showlegend=False,
            line_color=line_color_left,
            # hovertext=text_list[:positive_position],
            # fill = "blue",
            line=dict(width=0.05),
            stackgroup="1",
        ),
        row=1,
        col=j + 1,
    )
    distribution_figure.add_trace(
        go.Scatter(
            x=X[lower_bound_position:upper_bound_position],
            y=fx_list[lower_bound_position:upper_bound_position],
            hoverinfo="skip",
            # marker_symbol="square",
            name="< 0 ",
            marker=dict(size=50),
            showlegend=False,
            line_color=line_color_middle,
            # hovertext=text_list[:positive_position],
            # fill = "blue",
            line=dict(width=0.05),
            stackgroup="1",
        ),
        row=1,
        col=j + 1,
    )
    distribution_figure.add_trace(
        go.Scatter(
            x=X[upper_bound_position:],
            y=fx_list[upper_bound_position:],
            hoverinfo="skip",
            # marker_symbol="square",
            name="< 0 ",
            marker=dict(size=50),
            showlegend=False,
            line_color=line_color_right,
            # hovertext=text_list[:positive_position],
            # fill = "blue",
            line=dict(width=0.05),
            stackgroup="1",
        ),
        row=1,
        col=j + 1,
    )
    distribution_figure.add_trace(
        go.Scatter(
            x=X,
            y=fx_list,
            hoverinfo="text",
            # fill='tozeroy',
            mode="lines",
            # name = "+" ,
            showlegend=False,
            line_color="black",
            hovertext=text_list,
            line=dict(width=1.5),
        ),
        row=1,
        col=j + 1,
    )

    distribution_figure.update_xaxes(
        title_text=x_axis_title,
        row=1,
        col=j + 1,
    )


tick_values = [950, 1000, 1050]

distribution_figure.update_xaxes(
    zeroline=False,
    tickangle=0,
    # title = "" if x_axis_title is None else x_axis_title,
    tickvals=tick_values,
    title_font=dict(size=12),
    ticks="inside",  # Default is 'outside'
    ticklen=6,
)
distribution_figure.update_yaxes(
    title="",
    visible=False,
    showticklabels=False,
)
distribution_figure.update_traces(hoverlabel=dict(font=dict(size=18)))


distribution_figure.update_layout(height=300)
distribution_figure.update_layout(
    annotations=[
        dict(
            text="950 conversions or less",
            x=0.15,
            y=1.05,
            font=dict(size=12),
            showarrow=False,
        ),
        dict(
            text="1050 conversions or more",
            x=0.5,
            y=1.05,
            font=dict(size=12),
            showarrow=False,
        ),
        dict(
            text="Between 950 and 1050 conversions",
            x=0.85,
            y=1.05,
            font=dict(size=12),
            showarrow=False,
        ),
    ]
)


distribution_figure.show()

Once you've simulated your dataset, it is _**effortless**_ to calculate these probabilities!!!


In [ ]:
#| code-fold: false
#| echo: true
prob = mean(
    where(
        (control_simulated_results > 950)
        & (control_simulated_results < 1050),
        1,
        0,
    )
)
print(f"{prob:.2%}")

## Verify Simulation Results

Because Rate-based metrics are normally distributed, we can use the normal approximation to verify our simulation results are reasonable using mean = `{python} conversion_rate` and standard_error = `{python} f"({conversion_rate}"` * `{python} f"{1 - conversion_rate})/sqrt(10,000)"`.

$$
F(0.095) = \Phi\left( \frac{0.095 - 0.10}{0.003} \right) = \Phi\left( -1.67 \right) = 0.0478
$$


In [ ]:
# | code-fold: true
# | echo: false
import pandas as pd

probability_list = [f"{value:.2%}" for value in probability_list]
df = pd.DataFrame({"Type":distribution_titles, "Probability": probability_list})
print(df)

Slight differences are explained by the continuous nature of the normal approximation. Our data is discrete.


# Conclusion

By simulating 1MM simulations for our website conversion example, we were able to observe patterns and trends that might be difficult or costly to capture in real-world scenarios. For now, you should have learned how to use NumPy to generate rate-based simulated data and how to interpret the results.

This approach will prove even more valuable when dealing with problems that lack closed-form analytical solutions. 
We will demonstrate this utility in future topics when we expand our use of NumPy to model more complex AB Testing topics like:

- [**Experiment Design & Analysis with NumPy Random**: Understanding P-values and Statistical Power with NumPy Simulations](https://rzaliznyak-math.github.io/numpy-random-examples/){target="_blank"}.
- Continuous Metrics
- Sequential & Bayesian Testing
- Using Prior Information to Improve Effect Size Estimates
- CUPED: Using Covariates to Reduce Experimentation Run-time

# Acknowledgements

Big THANK YOU to my colleague Joseph Powers, PhD, who introduced me to _Quarto_ and using simulation studies to make my work life easier.